Re-running the graph witness notebook with refractored code

In [ ]:
import qcdenoise as qcd
import matplotlib.pyplot as plt
import qiskit as qk
import numpy as np
import networkx as nx

import os

In [ ]:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

### 1. Building Quantum Circuit from Graph States Database


#### Load the default graph state database

This is the corpus from which subgraphs are drawn and combined to form random graphs

In [ ]:
graph_db = qcd.GraphDB(directed=True)
graph_db.plot_graph(graph_number=[35])
graph_db['35']

In [ ]:
graph_db.plot_graph(graph_number=[17])
graph_db['17']

In [ ]:
graph_db.plot_graph(graph_number=[3])
graph_db['3']

#### Initiate a q-circuit builder from graph states

Build a random graph on 4 qubits with undirected edges

In [ ]:
n_qubits=4
graph_state = qcd.GraphState(graph_db=graph_db, n_qubits=n_qubits)
g = graph_state.sample()
nx.draw(g)

In [ ]:
circ_builder = qcd.CXGateCircuit(n_qubits=n_qubits,stochastic=True)

In [ ]:
circ_builder.stochastic

In [ ]:
circ_builder.build(g)["circuit"].draw(output='mpl')

In [ ]:
base_circuit = circ_builder.circuit

Once the circuit builder is initialized, and the base circuit is constructed- then the generators and stabilizer operators can be built

Calling `TothStabilizer` or `JungStabilizer` will build the sub-circuits needed to measure each stabilizer (Pauli string)

**Note** the generators and stabilizers will depend on the neighborhoods of each vertex and what is defined as a neighbor is dependent on the `NetworkX` graph structure.  If the edges are directed (pass `directed=True` inside `GraphConstructor`) then only vertices connected by a directed arc that ternminates at vertex (i) is a neighbor of (i).  On the other hand, if the graph is undirected (the default is `directed=False`) then any vertex connected by an edge to vertex (i) is a neighbor.

In [ ]:
stabilizer = qcd.TothStabilizer(g, n_qubits=n_qubits)

In [ ]:
stab_ops = stabilizer.find_stabilizers()

In [ ]:
stabilizer_circuit_dict = stabilizer.build()

In [ ]:
stabilizer_circuit_dict

The keys of the `circuit_dict` are the associated Pauli strings to measure-- comparing the keys to the graph `g` above, they are correctly defined

In [ ]:
stabilizer_circuit_dict['ZXIZ'].draw(output='mpl')

In [ ]:
stabilizer_circuit_dict['IIXZ'].draw(output='mpl')

In [ ]:
stabilizer_circuit_dict['IIII'].draw(output='mpl')

In [ ]:
stabilizer_circuit_dict['IZZX'].draw(output='mpl')

In [ ]:
stabilizer_circuit_dict['XZII'].draw(output='mpl')

_there is something wrong with the stabilizer construction the initial Pauli operator is being dropped_

**Current Workaround** when the stabilizer strings are built using `find_stabilizers` followed by `get_unique_stabilizers`, the leading sign coefficient is dropped, using `drop_coef=True` will then result in a Pauli operator getting dropped.  Current workaround sets the default value of `drop_coef=False`.

Stabilizer stubs are correctly built with this workaround

In [ ]:
stabilizer_circuit_dict

##### Build and mesure stabilizer circuits

pass the dictionary of stabilier sub-circuits and the base circuit

In [ ]:
from qiskit.test.mock import FakeValencia, FakeTokyo
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel

In [ ]:
backend = AerSimulator()
sampler = qcd.StabilizerSampler(
        backend=backend, n_shots=1024)

In [ ]:
ideal_counts = sampler.sample(stabilizer_circuits=stabilizer_circuit_dict,
                            graph_circuit=base_circuit)

In [ ]:
witness = qcd.GenuineWitness(n_qubits=n_qubits,
                             stabilizer_circuits=stabilizer_circuit_dict,
                             stabilizer_counts=ideal_counts)
witness.stabilizer_measurements


In [ ]:
witness.estimate(graph=g,noise_robust= 0)

In [ ]:
sampler.backend

In [ ]:
sampler.noise_model

In [ ]:
backend = AerSimulator.from_backend(FakeValencia())
sampler.backend=backend
noise_model=NoiseModel.from_backend(backend)
tokyo_counts =  sampler.sample(stabilizer_circuits=stabilizer_circuit_dict,
                            graph_circuit=base_circuit,noise_model=noise_model)

In [ ]:
sampler.noise_model

In [ ]:
witness = qcd.BiSeparableWitness(n_qubits=n_qubits,
                             stabilizer_circuits=stabilizer_circuit_dict,
                             stabilizer_counts=tokyo_counts)
witness.estimate(graph=g)



In [ ]:
witness.estimate(graph=g,noise_robust= 0)

#### Extract the expectation values from the counts
Use the built-in Qiskit functions of `ignis`.  when `Witness` is constructed, the diagonals and expectation values are evaluated and storedd.  Calling `evaluate()` constructs the witness value from these stored values  

In [ ]:
witness.stabilizer_circuits

In [ ]:
witness.stabilizer_counts

In [ ]:
witness.diagonals

In [ ]:
witness.stabilizer_measurements

In [ ]:
witness = qcd.GenuineWitness(n_qubits=n_qubits,
                             stabilizer_circuits=stabilizer_circuit_dict,
                            stabilizer_counts=tokyo_counts)
witness.stabilizer_measurements

#### Construct the witness value by hand using the stored values

In [ ]:
non_iden_keys = [x for x in witness.stabilizer_measurements if x!='IIII']
genuine_wit = (n_qubits-1)*witness.stabilizer_measurements['IIII'][0]-\
                    np.sum([witness.stabilizer_measurements[kdx][0] for kdx in non_iden_keys])

In [ ]:
genuine_wit

#### Construct the witness value using `.estimate()`

In [ ]:
witness.estimate(graph=g,noise_robust= 0)

In [ ]:
isinstance(backend, AerSimulator)